In [ ]:
import os
import glob
import torch
import gc

# # yolov5
---

## # yolov5 가져오기
---

In [ ]:
%cd /content/drive/MyDrive/workspace/study/3D_Object_detection
!git clone https://github.com/ultralytics/yolov5

## # 모듈 임포트
---

In [ ]:
%cd /data/NIA50/50-2/models/yolov5
!pip install -r requirements.txt  # install

## # 사전학습된 모델 다운로드
---

In [ ]:
import torch

# 사전 학습된 모델 뽑아내기
model = torch.hub.load('ultralytics/yolov5', 'yolov5l6')  # or yolov5n, yolov5x6, custom

## # 데이터에 맞춰 yaml 생성
---

In [ ]:
import glob
from sklearn.model_selection import train_test_split

img_list = sorted(glob.glob('/data/NIA50/data/2-050_sensor_sample/camera/front/images/*.jpg'))

# train_temp, test_list = train_test_split(img_list, test_size = 0.2, random_state = 0)

# train_list, val_list = train_test_split(train_temp, test_size = 0.2, random_state = 0)

train_list, val_list = train_test_split(img_list, test_size = 0.8, random_state = 0)


with open('/data/NIA50/yolov5/nia50/train.txt', 'w') as f:
    f.write('\n'.join(train_list) + '\n')
    
with open('/data/NIA50/yolov5/nia50/val.txt', 'w') as f:
    f.write('\n'.join(val_list) + '\n')
    
# with open('/data/NIA50/yolov5/nia50/test.txt', 'w') as f:
#     f.write('\n'.join(test_list) + '\n')
    
# with open('')

## # cuda 확인
---

In [ ]:
# gpu 캐시 삭제

import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
# gpu 할당

import os
import torch

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1' # gpu 여러개 할당

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## # 훈련
---

In [ ]:
# 훈련

%cd /home/ubuntu/kimgh/yolov5
!python train.py --img 640 --batch 16 --epochs 3 --data ./data/kitti.yaml --cfg ./models/yolov5x_kitti.yaml --weights yolov5x.pt --optimizer Adam --name kitti_result

In [ ]:
%cd /data/NIA50/yolov5
!python train.py --img 1280 --batch 16 --epochs 5 --data ./nia50/nia50.yaml --cfg ./nia50/yolov5l6.yaml --weights ./ckpt/yolov5l6.pt --optimizer Adam --name sample_result

In [ ]:
# NIA50_train_1st
%cd /data/NIA50/50-2/models/yolov5

data = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/nia50_train1st_data_yolov5l6.yaml'
model = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/nia50_train1st_model_yolov5l6.yaml'
# weights = '/data/NIA50/50-2/models/yolov5/runs/train/nia50_train1st2/weights/last.pt'
weights = '/data/NIA50/50-2/models/yolov5/ckpt/yolov5l6.pt'
name = 'nia50_train1st'

!python train.py --img 1200 --batch 12 --epochs 100 --data {data} --cfg {model} --weights {weights} --optimizer AdamW --name {name} --device 2,3

In [ ]:
# NIA50_train_1st
%cd /data/NIA50/50-2/models/yolov5

data = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/nia50_train1st_data_yolov5l6.yaml'
model = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/nia50_train1st_model_yolov5l6.yaml'
weights = '/data/NIA50/50-2/models/yolov5/runs/train/nia50_train1st/weights/best.pt'
name = 'nia50_train1st'

!python train.py --img 1200 --batch 16 --epochs 20 --data {data} --cfg {model} --weights {weights} --optimizer AdamW --name {name} --device 0,1,2,3

In [ ]:
%cd /data/NIA50/yolov5

data = '/data/NIA50/yolov5/nia50/yolo_aivill48/data_yolo_aivill48.yaml'
model = '/data/NIA50/yolov5/nia50/yolo_aivill48/model_yolov5l6.yaml'
weights = '/data/NIA50/yolov5/ckpt/yolov5l6.pt'

!python train.py --img 640 --batch 15 --epochs 100 --data {data} --cfg {model} --weights {weights} --optimizer AdamW --name yolo_avill48_re --device 1,2

In [ ]:
# temp_data
%cd /data/NIA50/50-2/models/yolov5

data = '/data/NIA50/50-2/data/NIA48/temp_data/temp_data_data_yolo5l6.yaml'
model = '/data/NIA50/50-2/data/NIA48/temp_data/temp_data_cfg_yolov5l6.yaml' # cfg
weights = '/data/NIA50/50-2/models/yolov5/ckpt/yolov5l6.pt'

!python train.py --img 1200 --batch 15 --epochs 100 --data {data} --cfg {model} --weights {weights} --optimizer AdamW --name temp_data --device 1,2,3

In [ ]:
%cd /data/NIA50/50-2/models/yolov5

!python train.py --help

## # 검증
---

In [ ]:
# 검증

%cd /home/ubuntu/kimgh/yolov5
!python val.py --weights ./runs/train/kitti_result3/weights/best.pt --data ./data/kitti.yaml --img 640

In [ ]:
%cd /data/NIA50/yolov5
!python val.py --data ./nia50/nia50.yaml --weights ./ckpt/yolov5l6.pt --batch-size 15 --img 1280 --verbose --name sample_val --save-txt --save-conf --exist-ok --device 0,1,2

In [ ]:
%cd /data/NIA50/50-2/models/yolov5

data = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/nia50_train1st_data_yolov5l6.yaml'
# model = '/data/NIA50/yolov5/nia50/yolo_aivill48/model_yolov5l6.yaml'
weights = '/data/NIA50/50-2/models/yolov5/runs/train/nia50_train1st3/weights/best.pt'
name = 'nia50_train1st'

!python val.py --data {data} --weights {weights} --batch-size 16 --img 1200 --verbose --name {name} --single-cls --save-txt --save-conf --save-json --exist-ok --device 0,1,2,3

In [ ]:
%cd /data/NIA50/50-2/models/yolov5

!python val.py --help

## # 테스트
---

In [1]:
%cd /data/NIA50/50-2/models/yolov5

import os
import glob

scenes = sorted(os.listdir('/data/NIA50/50-2/data/NIA50/학습용데이터_pre4/Suwon/labels'))
scenes = sorted(list(set([i[:-9] for i in scenes])))

data = 'data/nia50_data_yolov5l6.yaml'
# model = 'data/nia50_model_yolov5l6.yaml'
weights = '/data/NIA50/kimgh/docker/yolov5/ckpt/nia50_bestweights_yolov5l6.pt'
img_size = '1200 1920'
project = 'runs/nia50_train1st'

for scene in scenes[1:]:
    source = f'/data/NIA50/50-2/data/NIA50/train_1st/raw/{scene}/camera/camera_0'
    !python detect.py --weights {weights} --data {data} --img {img_size} --conf 0.4 --project {project} --source {source} --name {scene} \
        --save-txt --save-conf --exist-ok --nosave --device 0,1,2,3

/data/NIA50/50-2/models/yolov5
detect: weights=['/data/NIA50/kimgh/docker/yolov5/ckpt/nia50_bestweights_yolov5l6.pt'], source=/data/NIA50/50-2/data/NIA50/train_1st/raw/Suwon_A_2210261635_0039/camera/camera_0, data=data/coco128.yaml, imgsz=[1200, 1920], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=0,1,2,3, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/nia50_train1st, name=Suwon_A_2210261635_0039, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-203-g6371de88 Python-3.10.8 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)
                                                              CUDA:1 (Tesla T4, 15110MiB)
                                                              CUDA:2 (Tesla T4, 15110MiB)
                                                              CUDA:3 (Tesla T4, 15110MiB)

Fusi

In [ ]:
%cd /data/NIA50/yolov5

path = '/content/drive/MyDrive/workspace/study/3D_Object_detection/OpenPCDet/data/kittimot/training/image_02'
folder_list = sorted(os.listdir(path))

for folder_name in folder_list[:1]:
    source_path = path + '/' + folder_name
    !python detect.py --weights ./weights/yolo5s_kitti.pt --img 640 --conf 0.4 --source {source_path} --name {'train/' + folder_name} --data ./data/kitti.yaml \
        --save-txt --save-conf --exist-ok --device 0

In [ ]:
%cd /data/NIA50/yolov5

source_path = '/data/NIA50/data/2-050_sensor_sample/camera/front/images'

weights = '/data/NIA50/yolov5/runs/train/yolo_avill48_re/weights/best.pt'
data = '/data/NIA50/yolov5/nia50/yolo_aivill48/data_yolo_aivill48.yaml'

!python detect.py --weights {weights} --img 960 1280 --conf 0.4 --source {source_path} --name sample_yolo_avill48_re --data {data} \
        --save-txt --save-conf --exist-ok --device 0,1,2,3

In [ ]:
%cd /data/NIA50/50-2/models/yolov5

weights = '/data/NIA50/50-2/models/yolov5/runs/train/nia50_train1st3/weights/best.pt'
img_size = '1200 1920'
source_path = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/ImageSets'
data = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/nia50_train1st_data_yolov5l6.yaml'
name = 'nia50_train1st'

!python detect.py --weights {weights} --conf 0.4 --source {source_path} --data {data} --name {name} \
        --save-txt --save-conf --exist-ok --device 0,1,2,3

In [ ]:
%cd /data/NIA50/50-2/models/yolov5

source_path = '/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_2210261635_0001_calib_ok/camera/camera_0'
weights = '/data/NIA50/50-2/models/yolov5/ckpt/yolov5l6.pt'
img_size = '1200 1920'
name = 'NIA50_test'

!python detect.py --weights {weights} --img {img_size} --conf 0.4 --source {source_path} --name {name} --classes 0 1 2 3 5 6 7 \
        --save-txt --save-conf --exist-ok --device 0,1,2

In [1]:
%cd /data/NIA50/50-2/models/yolov5

!python detect.py --help

/data/NIA50/50-2/models/yolov5
usage: detect.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE]
                 [--data DATA] [--imgsz IMGSZ [IMGSZ ...]]
                 [--conf-thres CONF_THRES] [--iou-thres IOU_THRES]
                 [--max-det MAX_DET] [--device DEVICE] [--view-img]
                 [--save-txt] [--save-conf] [--save-crop] [--nosave]
                 [--classes CLASSES [CLASSES ...]] [--agnostic-nms]
                 [--augment] [--visualize] [--update] [--project PROJECT]
                 [--name NAME] [--exist-ok] [--line-thickness LINE_THICKNESS]
                 [--hide-labels] [--hide-conf] [--half] [--dnn]
                 [--vid-stride VID_STRIDE]

options:
  -h, --help            show this help message and exit
  --weights WEIGHTS [WEIGHTS ...]
                        model path or triton URL
  --source SOURCE       file/dir/URL/glob/screen/0(webcam)
  --data DATA           (optional) dataset.yaml path
  --imgsz IMGSZ [IMGSZ ...], --img IMGSZ [IM

## # 프레임 라벨 하나로 합치기
---

In [ ]:
# import glob
# from sklearn.model_selection import train_test_split

# img_list = sorted(glob.glob('/data/hwang/datasets/kitti/training/images/*.png'))

# train_list, val_list = train_test_split(img_list, test_size = 0.2, random_state = 0)

# with open('/home/ubuntu/kimgh/yolov5/kitti_yaml/train.txt', 'w') as f:
#     f.write('\n'.join(train_list) + '\n')
    
# with open('/home/ubuntu/kimgh/yolov5/kitti_yaml/val.txt', 'w') as f:
#     f.write('\n'.join(val_list) + '\n')

    
# # with open('')

In [ ]:
import pandas as pd
import re
import os

label_path = '/content/drive/MyDrive/workspace/study/3D_Object_detection/yolov5/runs/detect/train/0000/labels/'
frame_list = sorted(os.listdir(label_path))

label_df = pd.DataFrame()
for frame in frame_list:
    frame_num = int(re.search('[0-9]*', frame)[0])
    frame_df = pd.read_csv(label_path + frame, header = None, sep = ' ').replace(0, f'{frame_num}', regex = True)
    # frame_df = frame_df.replace(' ', ',', regex = True)

    label_df = pd.concat((label_df, frame_df), axis = 0)

label_df.to_csv('/content/drive/MyDrive/workspace/study/3D_Object_detection/yolov5/runs/detect/train/0000/0000.txt', index = None, header = None, sep = ',')

# # 로컬에서 모델 로드
---

In [ ]:
import cv2
import torch
import io
from PIL import Image

In [ ]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
weights = '/content/drive/MyDrive/workspace/study/3D_Object_detection/yolov5/weights/yolo5s_kitti.pt'

# model = torch.hub.load('ultralystics/yolov5', 'yolov5s') # 깃에서 직접 불러오기
model = torch.hub.load('/content/drive/MyDrive/workspace/study/3D_Object_detection/yolov5', 'custom', path = weights, source = 'local') # 로컬에서 불러오기

In [ ]:
path = '/content/drive/MyDrive/workspace/study/3D_Object_detection/OpenPCDet/data/kittimot/training/image_02'
folder_list = sorted(os.listdir(path))

img = path + '/' + folder_list[0] + '/000000.png'

In [ ]:
results = model(img)

In [ ]:
df = results.pandas().xyxy[0]
df

In [ ]:
a = df.iloc[0][:4].values.astype('float')
list(a)

In [ ]:
import numpy as np

a = np.array([554, 791, 605, 884]) / 1.6
a = np.array([554, 776, 749, 879]) / 1.6
a = np.array([871, 862, 1180, 956]) / 1.6
# a = np.array([1494, 801, 1883, 909]) / 1.6

In [ ]:
img = '/data/project50/1-cycle/camera/rear/1639543825.777566671.jpg'
img_.size

In [ ]:
from PIL import Image, ImageDraw, ImageFont

img_ = Image.open(img).convert('RGB')

color = (0,255,0)

font_size = 15
# font = ImageFont.truetype('/content/drive/MyDrive/workspace/study/3D_Object_detection/yolov5/fonts/arial.ttf', 25) # arial.ttf 글씨체, font_size=15
font = ImageFont.load_default()
# box_color_RGBA  = (0,255,0,255)
# fill_color_RGBA = (0,255,0,50)
# draw = ImageDraw.Draw(img_, 'RGBA') # RGBA
# draw.rectangle((100,100,300,300), outline=box_color_RGBA, fill=fill_color_RGBA, width = 3)
# img_.show()

draw = ImageDraw.Draw(img_)
text_pos = (a[0], a[1]-27)
# draw.text(text_pos, f"{df['name'][0], round(df['confidence'][0], 2)}", fill = color, font = font, stroke_width = 1)
draw.rectangle(list(a), outline = color, width = 3)
# img_.show()

img_